In [2]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
import os
import cv2
import matplotlib.pyplot as plt
import PIL.Image as img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
train_dir = 'drive/MyDrive/ML/Covid/Covid19-dataset/train'
test_dir = 'drive/MyDrive/ML/Covid/Covid19-dataset/test'

In [4]:
x_train = []
for folder in os.listdir(train_dir):
    sub_path = train_dir + '/' + folder
    for image in os.listdir(sub_path):
        img_path = sub_path + '/' + image
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224)) #VGG19 needs an input of shape 224x224
        x_train.append(img)

In [5]:
x_test = []
for folder in os.listdir(test_dir):
    sub_test = test_dir + '/' + folder
    for image in os.listdir(sub_test):
        img_test = sub_test + '/' + image
        imgtest = cv2.imread(img_test)
        imgtest = cv2.resize(imgtest,(224,224))
        x_test.append(imgtest)

In [ ]:
train = np.array(x_train)/255.0
test = np.array(x_test)/255.0

In [ ]:
train_data = ImageDataGenerator(rescale = 1/255.0 , validation_split=0.2)
test_data = ImageDataGenerator(rescale = 1/255.0)

In [ ]:
train_set = train_data.flow_from_directory(
                    train_dir,
                    subset = 'training',
                    batch_size=32,
                    target_size=(224,224),
                    class_mode = 'sparse'
)
val_set = train_data.flow_from_directory(
                    train_dir,
                    subset = 'validation',
                    batch_size=32,
                    target_size=(224,224),
                    class_mode = 'sparse'
)
test_set = test_data.flow_from_directory(
                    test_dir,
                    batch_size=32,
                    target_size=(224,224),
                    class_mode = 'sparse'
)

In [ ]:
y_train = train_set.classes 
y_val = val_set.classes
y_test = test_set.classes 
print(train_set.class_indices)

In [ ]:
plt.figure(figsize=(16,16))
for i in range(8):
    plt.subplot(2,4,i+1)
    plt.imshow(train[i])
    plt.title(y_train[i])

In [ ]:
vgg = VGG19(include_top = False , input_shape = (224,224,3),weights='imagenet')
for layer in vgg.layers :
    layer.trainable = False

In [ ]:
ES = tf.keras.callbacks.EarlyStopping(
    patience = 10,
    min_delta = 0.001,
    verbose = 0
)
model = tf.keras.Sequential([
    layers.RandomFlip(mode='horizontal'),
    layers.RandomRotation(factor=0.2),
    vgg,
    layers.Flatten(),
    layers.Dense(3, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
training = model.fit_generator(
    train_set,
    callbacks=[ES],
    steps_per_epoch = 7 ,
    epochs = 25,
    validation_data = val_set
)

In [ ]:
pip install Keras-Preprocessing

In [ ]:

# Part 4 - Making a single prediction
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
tf.__version__
from keras_preprocessing.image import load_img
import numpy as np
from matplotlib import pyplot as plt
from keras_preprocessing import image


def predict_image(imagepath, cnn):
    predict = image.load_img(imagepath, target_size = (224, 224))   
    predict_modified = image.img_to_array(predict)
    predict_modified = predict_modified / 255
    predict_modified = np.expand_dims(predict_modified, axis = 0)
    result = cnn.predict(predict_modified)
    if result[0][0]>= 0.5:
        prediction = 'covid'
        probability = result[0][0]
        print("probability = " + str(probability))
    elif result[0][1]>0.5:
        prediction = 'Normal'
        probability = 1 - result[0][0]
        print("probability = " + str(probability))
    else:
        prediction = 'Viral Pneumonia'
        probability = 1 - result[0][0]
        print("probability = " + str(probability))
    print("Prediction = " + prediction)
    return result
        


In [ ]:
filePath="drive/MyDrive/ML/Covid/Covid19-dataset/test/Covid/0100.jpeg"
R = predict_image(filePath,model)
test_image = image.load_img("drive/MyDrive/ML/Covid/Covid19-dataset/test/Covid/0100.jpeg", target_size = (224, 224))
plt.imshow(test_image)
R

In [ ]:
filePath="drive/MyDrive/ML/Covid/Covid19-dataset/test/Normal/0105.jpeg"
predict_image(filePath,model)
test_image = image.load_img("drive/MyDrive/ML/Covid/Covid19-dataset/test/Normal/0105.jpeg", target_size = (224, 224))
plt.imshow(test_image)



In [ ]:
filePath="drive/MyDrive/ML/Covid/Covid19-dataset/test/Viral Pneumonia/0106.jpeg"
predict_image(filePath,model)
test_image = image.load_img("drive/MyDrive/ML/Covid/Covid19-dataset/test/Viral Pneumonia/0106.jpeg", target_size = (224, 224))
plt.imshow(test_image)

